# Conectar al vostre "Google Drive"
Per entrenar un model de Tacotron2 necessitarem dades i un model de partida. La manera de accedir aquests fitxers és mitjançant dels enllaços de Google Drive afegir-los al vostre compte.

Un cop els fitxers estan a les nostres carpetes, podem donar els permisos a aquest "notebook" per accedir a les carpetes del drive. D'aquesta manera els fitxers seran visibles pel codi.

Els enllaços són:
* [Model de Tacotron2](https://drive.google.com/open?id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA
), entrenat amb LJ Speech
* [Dades de CMU Arctic](https://drive.google.com/open?id=1-DWmBkD99R09wEMb9r2MSD_l9qkgbNOf
); només la veu KSP, procecessat per Tacotron2
* [Model de Waveglow](https://drive.google.com/open?id=1WsibBTsuRg_SF2Z6L6NFRTT-NjEy1oTx), entrenat amb LJ Speech (per si de cas)

Després d'afegir el fitxer de tar al vostre drive hauríeu de descomprimir-lo. Assegureu que la vostra copia estigui a la "carpeta correcte."

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/tacotron_models"

In [0]:
!cp /content/drive/My\ Drive/data/adaptation_data.tar.gz /content/
!tar xzf adaptation_data.tar.gz
!rm adaptation_data.tar.gz

# Importar el codi
El "notebook" de colab ens deixa executar ordres del terminal d'un linux, mitjançant el `!` i `%`. A més, els servidors del colab venen amb certes aplicacions instal·lades com a CUDA i git.

Per importar el codi, farem un clon de github. Aquesta vegada farem servir el "fork" de Col·lectivaT perquè està especificament modificat per aquesta tasca.

In [0]:
import sys
!git clone https://github.com/collectivat-dev/catotron catotron
%cd catotron
!git submodule init; git submodule update
sys.path.append('/content/catotron/waveglow')

# Instal·lació de les llibreries i cridar-les

In [0]:
%%bash
pip install numpy scipy librosa unidecode inflect librosa tensorboardX

# Executar l'script d'entrenament

Aprofitarem l'script del repositori per l'entrenament. Fixeu-vos a l'opció de `--warm_start` que ens permet aprofitar del model entrenat.

També assegureu-vos que l'script està intentant accedint a les carpetes correctes.

In [0]:
!python train.py --output_directory="/content/drive/My Drive/tacotron_models/" --log_directory=/content/tacotron_adaptation/logs -c "/content/drive/My Drive/tacotron_models/tacotron2_statedict.pt" --warm_start

# Carregar els models i sintetitzar la veu

Després unes quantes iteracions de l'entrenament, ja podem investigar la qualitat dels resultats, sintetitzant algunes frases.

Per generar una veu, Tacotron2 necessita dos passos: el primer generar els mel espectrogrames i el segon generar les ones a partir dels espectrogrames. Per aquesta raó necessitem dos models un per Tacotron2 un altre pel Vocoder. En aquest cas un model de Waveglow.

In [0]:
# Generic libraries
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd
from scipy.io.wavfile import write

import numpy as np
import torch
from audio_processing import griffin_lim

# tacotron2 modules
from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT

import distributed
from train import load_model
from text import text_to_sequence
#from denoiser import Denoiser

In [0]:
hparams = create_hparams()
rate = 22050
hparams.sampling_rate = rate

checkpoint_path = "../drive/My Drive/tacotron_models/tacotron2_statedict.pt"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()

In [0]:
waveglow_path = '../drive/My Drive/tacotron_models/waveglow_256channels.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
#denoiser = Denoiser(waveglow)

# fix for the "AttributeError: 'ConvTranspose1d' object has no attribute 'padding_mode'"
for m in waveglow.modules():
    if 'Conv' in str(type(m)):
        setattr(m, 'padding_mode', 'zeros')

In [0]:
# introduce the text
text = "It's always darkest before it becomes totally black."

# preprocessing
sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
sequence = torch.from_numpy(sequence).to(device='cuda', dtype=torch.int64)

# run the models
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet)
audio_numpy = audio[0].data.cpu().numpy()

# make audio listenable
ipd.Audio(audio_numpy, rate=rate)


In [0]:
write('/content/drive/My Drive/test_wavs/nvidia01.wav', rate, audio_numpy)

# Generar gràfiques

Per donar una ullada a l'alineament i els espectrograma, fem servir matplotlib.

In [0]:
def plot_data(data, figsize=(16, 16)):
    fig, axes = plt.subplots(len(data), 1, figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none')
    plt.savefig('/content/result.png')

In [0]:
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))